In [5]:
from functools import reduce
import sys
flag_4_h1_dist = True

In [6]:
solution = [1,2,3,8,0,4,7,6,5]
def reverse_number(arr):
    reverse_number = 0
    for x in range(len(arr)):
        if arr[x] == 0:
            continue
        else:
            for y in range(x,len(arr)):
                if arr[y] > arr[x]:
                    reverse_number+=1
    return reverse_number

In [7]:
class Table:
    def __init__(self,arr):
        self.table = arr
    def empty(self):
        if len(self.table)==0:
            return False
        else:
            return True
    def pop(self):
        temp = self.table[0]
        self.table = self.table[1:]
        return temp
    def put(self,node):
        if len(self.table)==0:
            self.table = [node]
        elif node.value() <= self.table[0].value():
            self.table = [node] + self.table
        elif node.value() > self.table[-1].value():
            self.table = self.table + [node]
        else:
            for x in range(len(self.table)):
                if node.value() >= self.table[x].value() and node.value() <=self.table[x+1].value():
                    self.table = self.table[:x+1] + [node] + self.table[x+1:]
                    return
        return
    def have(self,_map):
        for x in range(len(self.table)):
            if str(self.table[x].map) == str(_map):
                return x
        return None

In [8]:
class Node:
    def __init__(self,_map):
        self.map = _map
        self.par = None
    def __str__(self):
        return reduce(lambda x,y:x+y,[str(self.map.arr[x]) for x in range(len(self.map.arr))])
    def add_par(self,Node):
        self.par = Node
    def height(self):
        if self.par == None:
            return 0
        else:
            return self.par.height()+1
    def value(self):
        if flag_4_h1_dist == True:
            return self.height() + self.map.h1_dist()
        else:
            return self.height() + self.map.h2_dist()

In [23]:
class _map:
    def __init__(self,arr):
        if arr ==None:
            print("error")
            self.arr = []
        else:
            self.arr = arr
        self.solution = solution
    def __len__(self):
        return len(self.arr)
    def h1_dist(self):
        #return reduce(lambda x,y:x+y,[1 if _map.arr[x]==solution.arr[x] else 0 for x in range(len(x))])
        sum_dist = 0
        for x in range(len(self.arr)):
            if self.arr[x] == 0:
                continue
            else:
                if self.arr[x] == self.solution[x]:
                    sum_dist+=1
        return 8-sum_dist
    def h2_dist(self):
        sum_dist = 0
        for x in range(len(self.arr)):
            if self.arr[x] == 0:
                continue
            else:
                target_index = self.solution.index(self.arr[x])
                sum_dist+= abs(target_index//3 - x//3)+abs(target_index%3-x%3)
        return sum_dist
    def switch(self,zero_index,target_index):
        arr = []
        for x in range(len(self.arr)):
            if x == zero_index:
                arr.append(self.arr[target_index])
            elif x == target_index:
                arr.append(0)
            else:
                arr.append(self.arr[x])
        return arr
    def alternative_moves(self):
        moves_arr = []
        print(self.arr)
        zero_index = self.arr.index(0)
        if zero_index //3 ==0:
            moves_arr.append(_map(self.switch(zero_index,zero_index+3)))
        elif zero_index //3 ==2:
            moves_arr.append(_map(self.switch(zero_index,zero_index-3)))
        elif zero_index//3 ==1:
            moves_arr.append(_map(self.switch(zero_index,zero_index-3)))
            moves_arr.append(_map(self.switch(zero_index,zero_index+3)))
        if zero_index%3==0:
            moves_arr.append(_map(self.switch(zero_index,zero_index+1)))
        elif zero_index%3==2:
            moves_arr.append(_map(self.switch(zero_index,zero_index-1)))
        else:
            moves_arr.append(_map(self.switch(zero_index,zero_index-1)))
            moves_arr.append(_map(self.switch(zero_index,zero_index+1)))
        return moves_arr
    def __str__(self):
        arr = [str(self.arr[x]) for x in range(len(self.arr))]
        s = ""
        for x in range(len(arr)):
            s+=str(arr[x])
            if x % 3==2:
                s+="\n"
        return s

In [20]:
class Astar:
    def __init__(self,arr,target_arr,flag_4_h1_star):
        print("__init__")
        self.flag_4_h1_star = flag_4_h1_star
        self.solution_node = _map(arr)
        self.root_node = _map(target_arr)
        self.root_Node = Node(self.root_node)
        self.open_table = Table([self.root_Node])
        self.close_table = Table([])
        self.tree_root =  self.root_Node
        self.h1_star = self.root_node.h1_dist()
        self.h2_star = self.root_node.h2_dist()
    def one_opt(self):
        current_node = self.open_table.pop()
        self.close_table.put(current_node)
        print("open table length {}".format(len(self.open_table.table)))
        print("close table length {}".format(len(self.close_table.table)))
        moves_arr = current_node.map.alternative_moves()
        print("{} possible moves".format(len(moves_arr)))
        for move in moves_arr:
            if str(move) == str(self.solution_node):
                temp = Node(move)
                temp.par = current_node
                return temp
        legit_moves = moves_arr
        print(legit_moves)
        '''
        if self.flag_4_h1_star == True:
            legit_moves = [x  for x in moves_arr if x.h1_dist()<=self.h1_star]
        else:
            legit_moves = [x  for x in moves_arr if x.h2_dist()<=self.h2_star]
        '''    
        for move in legit_moves:
            if self.open_table.have(move) != None:
                if current_node.height()+1 < self.open_table.table[self.open_table.have(move)].height():
                    self.open_table.table[self.open_table.have(move)].par = current_node
            elif self.close_table.have(move)!=None:
                if current_node.height()+1 < self.close_table.table[self.close_table.have(move)].height():
                    self.close_table.table[self.close_table.have(move)].par = current_node
            else:
                temp = Node(move)
                temp.par = current_node
                self.open_table.put(temp)
        return None

In [11]:
test = Astar([0,1,2,3,4,5,6,7,8],[1,0,2,3,4,5,6,7,8],True)

__init__


In [12]:
[test.one_opt() for x in range(1)]

open table length 0
close table length 1
3 possible moves


[None]

In [ ]:
if __name__ == "__main__":
    source_arr = []
    for x in range(3):
        arr = input("{} line:".format(x))
        arr = arr.split(" ")
        [source_arr.append(int(x)) for x in arr]
    print("source map is:",reduce(lambda x,y:x+" "+y+" ",[str(x) for x in source_arr]))
    if reverse_number(source_arr) %2 != reverse_number(solution)%2:
        print("no such solution")
        sys.exit(0)
    a_star_process = Astar(source_arr,solution,flag_4_h1_dist)
    while True:
        temp = a_star_process.one_opt()
        if temp!=None:
            while temp!=None:
                print(str(temp.map))
                temp = temp.par
                sys.exit(0)

0 line:1 2 3
1 line:4 5 6
2 line:8 7 0
source map is: 1 2  3  4  5  6  8  7  0 
__init__
open table length 0
close table length 1
[1, 2, 3, 8, 0, 4, 7, 6, 5]
4 possible moves
[<__main__._map object at 0x000001C1E42AB898>, <__main__._map object at 0x000001C1E42AB748>, <__main__._map object at 0x000001C1E42AB978>, <__main__._map object at 0x000001C1E42AB9E8>]
open table length 3
close table length 2
[1, 2, 3, 8, 4, 0, 7, 6, 5]
3 possible moves
[<__main__._map object at 0x000001C1E42AB8D0>, <__main__._map object at 0x000001C1E42ABA90>, <__main__._map object at 0x000001C1E42ABAC8>]
open table length 4
close table length 3
[1, 2, 3, 0, 8, 4, 7, 6, 5]
3 possible moves
[<__main__._map object at 0x000001C1E42ABAC8>, <__main__._map object at 0x000001C1E42ABB00>, <__main__._map object at 0x000001C1E42ABBA8>]
open table length 5
close table length 4
[1, 2, 3, 8, 6, 4, 7, 0, 5]
3 possible moves
[<__main__._map object at 0x000001C1E42ABBA8>, <__main__._map object at 0x000001C1E42ABBE0>, <__main__._

open table length 35
close table length 37
[1, 0, 4, 8, 3, 2, 7, 6, 5]
3 possible moves
[<__main__._map object at 0x000001C1E4294860>, <__main__._map object at 0x000001C1E42948D0>, <__main__._map object at 0x000001C1E4294908>]
open table length 35
close table length 38
[1, 3, 4, 8, 2, 5, 7, 6, 0]
2 possible moves
[<__main__._map object at 0x000001C1E4294860>, <__main__._map object at 0x000001C1E4294908>]
open table length 35
close table length 39
[8, 1, 3, 7, 2, 4, 0, 6, 5]
2 possible moves
[<__main__._map object at 0x000001C1E4294978>, <__main__._map object at 0x000001C1E42949B0>]
open table length 35
close table length 40
[1, 2, 0, 8, 6, 3, 7, 5, 4]
2 possible moves
[<__main__._map object at 0x000001C1E4269080>, <__main__._map object at 0x000001C1E42690B8>]
open table length 35
close table length 41
[0, 2, 3, 1, 6, 4, 8, 7, 5]
2 possible moves
[<__main__._map object at 0x000001C1E4269080>, <__main__._map object at 0x000001C1E4269160>]
open table length 35
close table length 42
[1, 2,

open table length 55
close table length 76
[1, 2, 3, 8, 5, 6, 0, 7, 4]
2 possible moves
[<__main__._map object at 0x000001C1E4259B38>, <__main__._map object at 0x000001C1E4259F28>]
open table length 55
close table length 77
[1, 2, 3, 6, 7, 4, 8, 5, 0]
2 possible moves
[<__main__._map object at 0x000001C1E4259F28>, <__main__._map object at 0x000001C1E4259A58>]
open table length 55
close table length 78
[0, 2, 3, 1, 7, 4, 6, 8, 5]
2 possible moves
[<__main__._map object at 0x000001C1E4259A58>, <__main__._map object at 0x000001C1E4259CC0>]
open table length 55
close table length 79
[2, 8, 3, 7, 1, 4, 0, 6, 5]
2 possible moves
[<__main__._map object at 0x000001C1E4259C18>, <__main__._map object at 0x000001C1E4259A58>]
open table length 55
close table length 80
[1, 2, 0, 8, 5, 3, 7, 4, 6]
2 possible moves
[<__main__._map object at 0x000001C1E4259C18>, <__main__._map object at 0x000001C1E4259A90>]
open table length 55
close table length 81
[1, 4, 2, 8, 3, 5, 7, 6, 0]
2 possible moves
[<__mai

8

In [28]:
s = input()
print(s[0])

1 2 3
1


2